# Test WHOIS MCP Server

This notebook demonstrates how to test the `whois_query` function from `whois_mcp_server.py`.

You can use this notebook to check WHOIS information for any domain.

In [4]:
import socket

def whois_query(domain: str) -> str:
    """Perform a WHOIS lookup for a domain name."""
    WHOIS_SERVER = "whois.iana.org"
    try:
        # Find the authoritative WHOIS server for the TLD
        with socket.create_connection((WHOIS_SERVER, 43), timeout=10) as s:
            s.sendall((domain + "\r\n").encode())
            response = b""
            while True:
                data = s.recv(4096)
                if not data:
                    break
                response += data
        response_text = response.decode(errors="ignore")
        # Find the "refer:" line for the authoritative server
        refer_line = next((line for line in response_text.splitlines() if line.lower().startswith("refer:")), None)
        if refer_line:
            whois_server = refer_line.split(":", 1)[1].strip()
        else:
            whois_server = WHOIS_SERVER

        # Query the authoritative WHOIS server
        with socket.create_connection((whois_server, 43), timeout=10) as s:
            s.sendall((domain + "\r\n").encode())
            response = b""
            while True:
                data = s.recv(4096)
                if not data:
                    break
                response += data
        return response.decode(errors="ignore")
    except Exception as e:
        return f"WHOIS lookup failed: {e}"


In [5]:
whois_query("appstore.com")  # Example usage

'   Domain Name: APPSTORE.COM\r\n   Registry Domain ID: 1776392_DOMAIN_COM-VRSN\r\n   Registrar WHOIS Server: whois.comlaude.com\r\n   Registrar URL: http://www.comlaude.com\r\n   Updated Date: 2024-07-14T23:09:10Z\r\n   Creation Date: 1998-08-14T04:00:00Z\r\n   Registry Expiry Date: 2025-08-13T04:00:00Z\r\n   Registrar: Nom-iq Ltd. dba COM LAUDE\r\n   Registrar IANA ID: 470\r\n   Registrar Abuse Contact Email: abuse@comlaude.com\r\n   Registrar Abuse Contact Phone: +442074218250\r\n   Domain Status: clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited\r\n   Domain Status: clientTransferProhibited https://icann.org/epp#clientTransferProhibited\r\n   Domain Status: clientUpdateProhibited https://icann.org/epp#clientUpdateProhibited\r\n   Name Server: A.NS.APPLE.COM\r\n   Name Server: B.NS.APPLE.COM\r\n   Name Server: C.NS.APPLE.COM\r\n   Name Server: D.NS.APPLE.COM\r\n   DNSSEC: unsigned\r\n   URL of the ICANN Whois Inaccuracy Complaint Form: https://www.icann.org/wicf/\r